In [ ]:
import pickle
from scipy.io import loadmat
import numpy as np
import pandas as pd
from time import time as t
import matplotlib.pyplot as plt
#!pip install tqdm
from tqdm import tqdm
#!pip install git+https://github.com/BindsNET/bindsnet.git

from bindsnet.network.nodes import IFNodes, Input #, IzhikevichNodes # , McCullochPitts, AdaptiveLIFNodes
from bindsnet.network import Network
from bindsnet.learning import PostPre
from bindsnet.network.monitors import Monitor
from bindsnet.network.topology import Connection, Conv2dConnection, MaxPool2dConnection
from bindsnet.analysis.plotting import (
    plot_input,
    plot_spikes,
    plot_conv2d_weights,
    plot_voltages,
)


In [ ]:
ttfs_spikes_train = pd.read_pickle(r'ttfs_spikes_data/ttfs_spikes_train.p')
# print(ttfs_spikes_train.shape) # (index, time-frame, frequency-band)

In [ ]:
def one_hot2spikes(data):
    """
    Computes one-hot encoding from data
    Returns: [samples, time-frames, frequency bands, time-points]
    """
    n_spiketime_bins = np.max(data).astype(int) + 1
    print(n_spiketime_bins)
    spikes = np.zeros(list(data.shape)+[n_spiketime_bins])
    for i, sample in enumerate(data):
        for j, tf in enumerate(sample):
            spikes[i, j] = np.eye(n_spiketime_bins)[tf]

    return spikes

data_one_hot = one_hot2spikes(ttfs_spikes_train.astype(int))

32


### Creating and adding layers

In [ ]:
# bindsnet/conv_mnist.py: https://github.com/BindsNET/bindsnet/blob/master/examples/mnist/conv_mnist.py
# encodings: https://github.com/BindsNET/bindsnet/blob/master/bindsnet/encoding/encodings.py
time = 500


network = Network(dt=1.0, batch_size=1, learning=True, reward_fn=None)  # 1.0, 1, True and None are default values

## Input layer (40 x 41)
inp = Input(shape=(40, 41,31), traces=True)
network.add_layer(layer=inp, name="Input Layer")

## Convolutional layer
conv_layer = IFNodes(
    # n=100,         # number of neurons
    shape=(40,41,31), # The dimensionality of the layer
    traces=True,   # Whether to record spike traces
    thresh=23.0,    # spike threshold voltage, default=-52.0
    reset=0.0,     # post-spike reset voltage, default=-65.0
)
conv_conn = Conv2dConnection(
    inp,
    conv_layer,
    kernel_size=(6,40),
    stride=1,
    update_rule=PostPre,  # this should be STDP (?)
    # norm=0.4 * kernel_size ** 2,
    # nu=[1e-4, 1e-2],
    # wmax=1.0,
)
network.add_layer(layer=conv_layer, name="Convolutional Layer")
network.add_connection(conv_conn, "Input Layer", "Convolutional Layer")

## Pooling layer
# pool_layer = some node that does not spike according to the paper
# pool_conn = MaxPool2dConnection(
#     source=,       # A layer of nodes from which the connection originates
#     target=,       # A layer of nodes to which the connection connects
#     kernel_size=,  # Horizontal and vertical size of convolutional kernels
#     stride=,       # Horizontal and vertical stride for convolution
#     padding=,      # Horizontal and vertical padding for convolution
#     dilation=,     # Horizontal and vertical dilation for convolution
# )
# network.add_layer(layer=pool_layer, name="Pooling Layer")
# network.add_connection(pool_conn, "Convolutional Layer", "Pooling Layer")


# Create a monitor.

layers = ['Input', 'Convolutional Layer'] # 'Pooling Layer'

for idx, layer in enumerate(network.layers.items()):
    monitor = Monitor(
        obj=layer,
        state_vars=("s", "v"),  # Record spikes and voltages.
        time=500,  # Length of simulation (if known ahead of time).
    )
    network.add_monitor(monitor=monitor, name=f"{layers[idx]}")




AssertionError: Target dimensionality must be (out_channels, ?,(input_height - filter_height + 2 * padding_height) / stride_height + 1,(input_width - filter_width + 2 * padding_width) / stride_width + 1

In [ ]:
# train


input_data = data_one_hot
inputs = {"Input": input_data}
network.run(inputs=inputs, time=time)